<a href="https://colab.research.google.com/github/manojgali/languageTranslator/blob/main/machinetranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving model_final.h5 to model_final.h5


In [ ]:
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
import re
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
from urllib import request
from bs4 import BeautifulSoup
import numpy as np
from pickle import load

In [ ]:
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
dataset = array(load_clean_sentences('english-telugu-both.pkl'))
train = array(('english-telugu-train.pkl'))
test = array(load_clean_sentences('english-telugu-test.pkl'))

In [ ]:
 #fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [ ]:
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

In [ ]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [ ]:
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [ ]:
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 300:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
 

In [ ]:
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare telugu tokenizer
tel_tokenizer = create_tokenizer(dataset[:, 1])
tel_vocab_size = len(tel_tokenizer.word_index) + 1
tel_length = max_length(dataset[:, 1])
print('telugu Vocabulary Size: %d' % tel_vocab_size)
print('telugu Max Length: %d' % (tel_length))

English Vocabulary Size: 5199
English Max Length: 9
telugu Vocabulary Size: 9581
telugu Max Length: 9


In [ ]:
test_x =  Tel[5000:6000]
test_y =  Eng[5000:6000]

In [ ]:
testX = encode_sequences(tel_tokenizer, tel_length, test[:, 1])

In [ ]:
testX.shape

(1000, 30)

In [ ]:
model = load_model('model_final.h5')

In [ ]:
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

test
src=[ఏ కారణానికి?], target=[For what purpose?], predicted=[don't it it]
src=[తలుపు లాక్ చేయబడిందని మీరు నిర్ధారించుకున్నారా?], target=[Have you made sure the door is locked?], predicted=[don't me me to a you]
src=[మీరు సరేనని ఖచ్చితంగా అనుకుంటున్నారా?], target=[Are you sure you're OK?], predicted=[do you have any the you]
src=[విద్య అనేది భవిష్యత్తులో పెట్టుబడి.], target=[Education is an investment in the future.], predicted=[he is a the the]
src=[ఫ్రెంచ్ మాట్లాడే ఎవరైనా మీకు తెలుసా?], target=[Do you know anyone who speaks French?], predicted=[do you know what to a a]
src=[అనువాదాన్ని అసలుతో పోల్చండి.], target=[Compare the translation with the original.], predicted=[a is the the the the]
src=[మీరు అక్కడ వ్యాపారానికి వెళ్తున్నారా?], target=[Are you going there on business?], predicted=[are you like me]
src=[అతని వద్ద తగినంత డబ్బు ఉంది.], target=[He had more than enough money.], predicted=[he has a a]
src=[అతను హోటల్ లోనే ఉన్నాడు.], target=[He stayed in the hotel.], predicted=[he is

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.138645
BLEU-2: 0.048432
BLEU-3: 0.029566
BLEU-4: 0.052197


In [ ]:
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

test
src=[పిల్లలు ప్రతిరోజూ పాలు తాగాలి.], target=[Children should drink milk every day.], predicted=[are you you]
src=[పిల్లలు బడికి వెళ్ళాలి.], target=[Children should go to school.], predicted=[are you]
src=[పిల్లలు నదికి దూరంగా ఉండాలి. ఇది ప్రమాదకరం.], target=[Children should keep away from the river. It's dangerous.], predicted=[are you you]
src=[పిల్లలు చెడిపోకూడదు.], target=[Children should not be spoiled.], predicted=[are are you]
src=[పిల్లలు తల్లిదండ్రులకు కట్టుబడి ఉండాలి.], target=[Children should obey their parents.], predicted=[are you]
src=[పిల్లలు మాట్లాడినప్పుడు మాత్రమే మాట్లాడాలి.], target=[Children should only speak when they're spoken to.], predicted=[are you]
src=[పిల్లలు బయట ఆడాలి.], target=[Children should play outside.], predicted=[are you]
src=[పిల్లలు చెడిపోకూడదు.], target=[Children shouldn't be spoiled.], predicted=[are are you]
src=[పిల్లలు వైన్ తాగకూడదు.], target=[Children shouldn't drink wine.], predicted=[are you]
src=[పిల్లలకు కొన్నిసార్లు సహనం ఉండదు.], t

In [ ]:
rows, cols = (len(test_x), 1) 
test=[] 
for i in range(rows): 
    col = [] 
    for j in range(cols): 
        col.append(test_y[i]) 
        col.append(test_x[i])
    test.append(col) 
 